In [ ]:
import boto3
import json
from datetime import datetime

In [ ]:
def create_pii_guardrail(bedrock_client, name, description):
    # Define the guardrail configuration
    guardrail_config = {
        "name": name,
        "description": description,
        "blockedInputMessaging": "This input contains sensitive personal information and cannot be processed.",
        "blockedOutputsMessaging": "This response contains sensitive information and has been blocked.",
        
        # Configure PII filtering with correct enum values
        "sensitiveInformationPolicyConfig": {
            "piiEntitiesConfig": [
                {
                    "type": "EMAIL",
                    "action": "BLOCK"
                },
                {
                    "type": "PHONE",
                    "action": "ANONYMIZE"
                },
                {
                    "type": "CREDIT_DEBIT_CARD_NUMBER",
                    "action": "BLOCK"
                },
                {
                    "type": "US_SOCIAL_SECURITY_NUMBER",
                    "action": "BLOCK"
                },
                {
                    "type": "ADDRESS",
                    "action": "ANONYMIZE"
                },
                {
                    "type": "NAME",  # Changed from PERSON_NAME to NAME
                    "action": "ANONYMIZE"
                }
            ]
        },
        
        # Content filtering with correct enum values
        "contentPolicyConfig": {
            "filtersConfig": [
                {
                    "inputStrength": "HIGH",
                    "outputStrength": "HIGH",
                    "type": "MISCONDUCT"
                },
                {
                    "inputStrength": "MEDIUM",
                    "outputStrength": "HIGH",
                    "type": "HATE"  # Changed from PRIVACY to HATE
                }
            ]
        },
        
        # Word filtering
        "wordPolicyConfig": {
            "wordsConfig": [
                {
                    "text": "ssn"
                },
                {
                    "text": "social security"
                },
                {
                    "text": "password"
                },
                {
                    "text": "secret"
                }
            ]
        },
        
        # Tags in correct format
        "tags": [
            {
                "key": "Environment",
                "value": "Production"
            },
            {
                "key": "Service",
                "value": "PizzaFeedback"
            }
        ]
    }
    
    try:
        # Create the guardrail with error handling
        response = bedrock_client.create_guardrail(**guardrail_config)
        print("Guardrail created successfully:")
        print(json.dumps(response, indent=2, default=str))
        return response['guardrailId']
    
    except bedrock_client.exceptions.ValidationException as e:
        print(f"Validation error: {str(e)}")
        raise
    except bedrock_client.exceptions.AccessDeniedException as e:
        print(f"Access denied: {str(e)}")
        raise
    except bedrock_client.exceptions.ConflictException as e:
        print(f"Conflict error: {str(e)}")
        raise
    except bedrock_client.exceptions.InternalServerException as e:
        print(f"Internal server error: {str(e)}")
        raise
    except bedrock_client.exceptions.ThrottlingException as e:
        print(f"Throttling error: {str(e)}")
        raise
    except bedrock_client.exceptions.TooManyTagsException as e:
        print(f"Too many tags error: {str(e)}")
        raise
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise


In [ ]:
def main():
    # Initialize the Bedrock client with your specific profile
    bedrock = boto3.Session(profile_name='stuartck-admin')
    
    # Create the bedrock client (not bedrock-runtime)
    bedrock_client = bedrock.client(service_name='bedrock')
    
    # Example usage
    guardrail_name = "PII_Protection_Guardrail"
    guardrail_description = "Guardrail to protect customer PII information"
    
    guardrail_id = create_pii_guardrail(bedrock_client, guardrail_name, guardrail_description)
    
    if guardrail_id:
        print("\nGuardrail configuration completed successfully")

In [ ]:
if __name__ == "__main__":
    main()